In [12]:
import requests


In [10]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-large-en-v1.5')
model = AutoModel.from_pretrained('BAAI/bge-large-en-v1.5').cuda()
model = model.eval()


In [11]:
chunks, embeds = torch.load("database.pt")

In [14]:
questions = ["What is group relative policy optimization", "What rewards are used in DeepSeek-R1-Zero"]
for query in questions:
    encoded_input = tokenizer(query, padding="longest", truncation=True, return_tensors='pt')
    for k, v in encoded_input.items():
        encoded_input[k] = v.cuda()
    with torch.no_grad():
        model_output = model(**encoded_input)
        sentence_embeddings = model_output[0][:, 0]
        sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)
        topk = (sentence_embeddings @ embeds.T).topk(10)
        chunk = chunks[topk.indices[0][0]]
        
    url = "http://localhost:8000/v1/chat/completions"
    headers = {
        "Content-Type": "application/json"
    }
    data = {
        "model": "microsoft/Phi-3-mini-128k-instruct",
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "system", "content": chunk},
            {"role": "user", "content": query}
        ]
    }
    
    response = requests.post(url, headers=headers, json=data)
    print(query)
    print(response.json()["choices"][0]["message"]["content"])
    

What is group relative policy optimization
 Group relative policy optimization (GRPO) is an approach designed to address the high computational cost and complexity associated with training traditional reinforcement learning (RL) algorithms, namely actor-critic methods. These methods typically involve maintaining a critic model that is on par in size with the policy model, making each training iteration expensive and time-consuming.

To cut down on training costs, GRPO adopts an innovative strategy. It samples a group of outputs (answer records) — denoted as {𝑜1, 𝑜2, · · · , 𝑜𝐺} — from the old policy model denoted by 𝜃𝜑𝒯𝒸 and optimizes the policy model 𝜋𝜃 by maximizing the following objective:

J𝐺𝑅𝑃𝑂(𝜃) = E[𝑞∼𝑃(𝑄), {𝑜𝑖}𝐺
𝑖=1 ∼𝜋𝜃𝑜𝑙𝑑(𝑂|𝑄)]

In this expression, 
𝜃 represents parameters of 𝜑𝜑𝜑𝜑𝑃
𝑄 represents labels (questions)
𝑂 represents output records (responses)
𝑈𝑍 represents utterances
𝑖 represents the iteration in sampling from 𝑋𝑌 = {𝑂1, 𝑂2, ... , 𝑂𝐺}
𝜋𝜃𝑜𝑙𝑑 denotes the policy model an

torch.return_types.topk(
values=tensor([[0.6336, 0.6263, 0.6240, 0.6224, 0.6200, 0.6142, 0.6137, 0.6088, 0.6086,
         0.6046]], device='cuda:0'),
indices=tensor([[106,  25,   8,  13,  40,  22,  41, 107, 102,  42]], device='cuda:0'))

In [9]:

url = "http://localhost:8000/v1/chat/completions"
headers = {
    "Content-Type": "application/json"
}
data = {
    "model": "microsoft/Phi-3-mini-128k-instruct",
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "system", "content": chunks[41]},
        {"role": "user", "content": "what is LLaDA?"}
    ]
}

response = requests.post(url, headers=headers, json=data)

print(response.json()["choices"][0]["message"]["content"])

 LLaDA (Large Language Diffusion Model) is an innovative framework presented for large language modeling. Its unique approach stems from utilizing diffusion models, which are part of a class of generative models that iteratively transform a distribution of noise into a coherent output or data sample. LLaDA stands out because it is the first to apply these diffusion models to natural language processing effectively. 


The main contributions and features of LLaDA include:


1. **Scalability:** LLaDA can be scaled up to handle larger datasets and generate more complex language patterns due to its diffusion-based architecture.


2. **In-context Learning:** This denotes the model's ability to understand and generate responses based on the context provided by the preceding text. LLaDA demonstrates this capability quite effectively.


3. **Instruction Following:** LLaDA can perform tasks that require understanding complex instructions and generating appropriate responses, challenging the per

In [18]:
sentence_embeddings.shape

torch.Size([1, 1024])

In [13]:
len(chunks)

110

In [14]:
embeds.shape

torch.Size([1512, 1024])

In [ ]:
par